In [1]:
import graphlab

In [51]:
#input files
dir_csv="/Users/aponsero/Documents/UA_POSTDOC/projects/B2_project/MongoDB_ingest/test_passive_samplers/samples"
dir_voc="/Users/aponsero/Documents/UA_POSTDOC/projects/B2_project/b2_drought/vocabulary"
file_csv=dir_csv+"/mapped_convert_samples_leaves.csv"
vocabulary_file=dir_voc+"/specimen_controlled_vocabulary.csv"
typology_file=dir_voc+"/specimen_typology.csv"

#output file
out_dir="/Users/aponsero/Documents/UA_POSTDOC/projects/B2_project/MongoDB_ingest/test_passive_samplers/samples"
json_file=out_dir+"/samples_leaves.json"
fout = open(json_file, "a")

#sample type
my_sample_type="live_leaf_sample"

In [52]:
my_frame=graphlab.SFrame.read_csv(file_csv, column_type_hints=str)

Finished parsing file /Users/aponsero/Documents/UA_POSTDOC/projects/B2_project/MongoDB_ingest/test_passive_samplers/samples/mapped_convert_samples_leaves.csv

Parsing completed. Parsed 51 lines in 0.036015 secs.

In [64]:
#minimum information checklist
my_typo=graphlab.SFrame.read_csv(typology_file, column_type_hints=str)

minimum_tab=my_typo[(my_typo['Types'] == my_sample_type)]
minimum_fields=minimum_tab['required_fields']

for x in minimum_fields:
    if x in my_frame.column_names():
        print(x+": OK")
    else :
        print(x+" : not found")


check =  all(elem in my_frame.column_names() for elem in minimum_fields)
 
if check:
    print("The document passes the checklist")    
else :
    print("Warning : minimum attributes missing !")

Finished parsing file /Users/aponsero/Documents/UA_POSTDOC/projects/B2_project/b2_drought/vocabulary/specimen_typology.csv

Parsing completed. Parsed 89 lines in 0.083158 secs.

sample_ID not found
sample_type: OK
collection_protocol_ID: OK
sampling_location_on_grid: OK
sampling_location_height_level: OK
date-Time_of_collection not found
date_of_collection: OK
time_of_collection: OK
tree_host: OK
tree_ID: OK
sun_exposure: OK
sampling_location_height_from_the_ground: OK
sampling_location_closest_frame_level: OK
leaf_age: OK
operator not found
No, list1 does not contains all elements in list2


In [24]:
my_voc=graphlab.SFrame.read_csv(vocabulary_file, column_type_hints=str)
my_voc

Finished parsing file /Users/aponsero/Documents/UA_POSTDOC/projects/B2_project/b2_drought/vocabulary/specimen_controlled_vocabulary.csv

Parsing completed. Parsed 31 lines in 0.018471 secs.

Term,Displayed_term,Definition,Section_object,Units
sample_ID,sample ID,Unique identifier for thesample. ...,Specimen_description,
sample_type,sample type,type of sample. Followsone of the types of ...,Specimen_description,
collection_protocol_ID,collection protocol ID,ID for the protocol usedduring the sampling e ...,Collection,
tree_host,tree host,"For sample of type ""liveleaf sample"" or ""root ...",Location,
tree_ID,tree ID,"For sample of type ""liveleaf sample"" or ""root ...",Location,
sampling_location_on_grid,sampling location on grid,Original location on the2D grid system of the ...,Location,
sampling_location_height_level ...,sampling location heightlevel ...,Original location heightlevel of the sample ...,Location,
sample_replicate_number,sample replicate number,Number of the currentexperimental replicate ...,Specimen_description,
time_in_plastic_enclosure_before_collection ...,time in plastic enclosurebefore collection (in ...,Number of days spent inthe enclosure before ...,Collection,days
sample_treatment_before_collection ...,sample treatment beforecollection ...,Treatment performed onthe sample before ...,Collection,


In [37]:
# find the object categories and create type finder
Specimen_description=[]
Collection=[]
Location=[]
No_object=[]
attribute_types={}

for i in my_frame.column_names():
    curr_data=my_voc[(my_voc['Term'] == i)]
    section=curr_data['Section_object']
    att_type=curr_data['Type']
    #add type in attribute_types
    attribute_types[i] = att_type[0]
    
    #add the attribute in the object categories
    if section[0] == "Specimen_description":
        Specimen_description.append(i)
    elif section[0] == "Collection":
        Collection.append(i)
    elif section[0] == "Location":
        Location.append(i)
    else:
        No_object.append(i)
        
print(Specimen_description)
print(Collection)
print(Location)
print(No_object)

print(attribute_types)

['sample_type', 'sample_replicate_number', 'sample_number_displayed_on_tag', 'sample_name', 'Leaf_condition_before_enclosure', 'leaf_age', 'leaf_condition_after_enclosure', 'notes']
['collection_protocol_ID', 'time_in_plastic_enclosure_before_collection', 'sample_treatment_before_collection', 'sampler_size', 'date-time_of_enclosure_placement', 'date_of_enclosure_placement', 'time_of_enclosure_placement', 'date-time_of_collection', 'date_of_collection', 'time_of_collection']
['tree_host', 'tree_ID', 'sampling_location_on_grid', 'sampling_location_height_level', 'sun_exposure', 'sampling_location_height_from_the_ground', 'sampling_location_closest_frame_level']
[]
{'sampler_size': 'string', 'sampling_location_height_from_the_ground': 'num', 'leaf_condition_after_enclosure': 'string', 'Leaf_condition_before_enclosure': 'string', 'time_in_plastic_enclosure_before_collection': 'num', 'sampling_location_closest_frame_level': 'num', 'date_of_enclosure_placement': 'date', 'sample_type': 'strin

In [39]:
nb_samples=my_frame.shape[0]

for i in range(0,nb_samples):
    curr_slice=my_frame[i]
    line="{"
    #process the specimen description
    for i in range(len(Specimen_description)):
        Key=Specimen_description[i]
        if attribute_types[Key]=="int" or attribute_types[Key]=="float" or attribute_types[Key]=="num":
            attribute=str(curr_slice[Specimen_description[i]])
        elif attribute_types[Key]=="date-time":
            if str(curr_slice[Specimen_description[i]])=="NA" or str(curr_slice[Specimen_description[i]])=="None":
                attribute="\"None\""
            else:
                attribute="{$date:\""+str(curr_slice[Specimen_description[i]])+"\"}"
        else:
            attribute="\""+str(curr_slice[Specimen_description[i]])+"\""
        line=line+"\""+Key+"\":"+attribute+","

    #print("#### "+line)
    
    #process the Location description
    line=line+"\"Location\":{"
    for i in range(len(Location)):
        Key=Location[i]
        if attribute_types[Key]=="int" or attribute_types[Key]=="float":
            attribute=str(curr_slice[Location[i]])
        elif attribute_types[Key]=="dateTime":
            if str(curr_slice[Location[i]])=="NA" or str(curr_slice[Location[i]])=="None":
                attribute="\"None\""
            else:
                attribute="{$date:\""+str(curr_slice[Location[i]])+"\"}"
        else:
            attribute="\""+str(curr_slice[Location[i]])+"\""
        line=line+"\""+Key+"\":"+attribute
        
        if i==len(Location)-1:
            line=line+"},"
        else:
            line=line+","
    #print("#### "+line)

    #process the Collection description
    line=line+"\"Collection\":{"
    for i in range(len(Collection)):
        Key=Collection[i]
        if attribute_types[Key]=="int" or attribute_types[Key]=="float":
            attribute=str(curr_slice[Collection[i]])
        elif attribute_types[Key]=="dateTime":
            if str(curr_slice[Collection[i]])=="NA" or str(curr_slice[Collection[i]])=="None":
                attribute="\"None\""
            else:
                attribute="{$date:\""+str(curr_slice[Collection[i]])+"\"}"
        else:
            attribute="\""+str(curr_slice[Collection[i]])+"\""
        line=line+"\""+Key+"\":"+attribute
        
        if i==len(Collection)-1:
            line=line+"}"
        else:
            line=line+","
    
    
    line=line+"}"
    fout.write(line+"\n")
    print(line)
    

{"sample_type":"live_leaf_sample","sample_replicate_number":2,"sample_number_displayed_on_tag":"11","sample_name":"C1-H1-R2-2U","Leaf_condition_before_enclosure":"G","leaf_age":"m","leaf_condition_after_enclosure":"Nothing to note","notes":"None","Location":{"tree_host":"Clitoria","tree_ID":"1","sampling_location_on_grid":"5;0","sampling_location_height_level":"1","sun_exposure":"shade","sampling_location_height_from_the_ground":"10.5","sampling_location_closest_frame_level":"0"},"Collection":{"collection_protocol_ID":"2","time_in_plastic_enclosure_before_collection":"2","sample_treatment_before_collection":"Unwashed","sampler_size":"S","date-time_of_enclosure_placement":"2019-03-21T12:00:00-07:00","date_of_enclosure_placement":"3/21/19","time_of_enclosure_placement":"12:00","date-time_of_collection":"2019-03-23T11:30:00-07:00","date_of_collection":"23-Mar","time_of_collection":"11:30"}}
{"sample_type":"live_leaf_sample","sample_replicate_number":3,"sample_number_displayed_on_tag":"12"

In [42]:
fout.close()